In [0]:
import os
import numpy as np
import pandas as pd
import gc

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [4]:
!pip install transformers==2.6.0

     |████████████████████████████████| 542kB 3.2MB/s 
     |████████████████████████████████| 3.7MB 8.2MB/s 
     |████████████████████████████████| 870kB 38.7MB/s 
     |████████████████████████████████| 1.0MB 43.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e7f89d7c2d2b7916b44434d4fd89df72677ce8adad5cf13e1bc284fe8c832325
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
!pip install tensorflow==2.2.0rc2

In [6]:
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
import traitlets
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from sklearn.metrics import roc_auc_score
from tensorflow.keras import backend as K
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score

warnings.simplefilter("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [8]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')

# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer("vocab.txt", lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)

In [0]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [0]:
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #x = tf.keras.layers.Dropout(0.35)(cls_token)
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=3e-5),
                  metrics=[tf.keras.metrics.AUC()])
    
    return model

In [0]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
fsc = []

In [0]:
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models_toxic'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models_toxic'

In [0]:
TRAIN = False

In [15]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    
    # Tokenization within the CV to ensure not having overfitting
    
    X_train = fast_encode(X_train, fast_tokenizer, maxlen=400)
    X_val = fast_encode(X_val, fast_tokenizer, maxlen=400)

    train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train, y_train))
    .shuffle(42)
    .batch(64)
    .prefetch(AUTO)
    )

    valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_val, y_val))
    .batch(64)
    .cache()
    .prefetch(AUTO)
    )
    
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    with strategy.scope():
      transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
      model = build_model(transformer=transformer_layer,  max_len=400)
    

    #model.layers[1].trainable = False
    model.summary()

    print('Fitting')
    
    if TRAIN:
      history              = model.fit(train_dataset,
                                      epochs=1
                                      )
      model.save_weights(f'{cv_models_path}/DistilBERT_full_fold_{c}.h5')
    else:
      model.load_weights(f'{cv_models_path}/DistilBERT_full_fold_{c}.h5')

    probs                = model.predict(valid_dataset, batch_size=64, verbose=1)
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')

    threshold = 0.1
    probs_class = probs.copy()
    probs_class[probs_class >= threshold] = 1 
    probs_class[probs_class < threshold] = 0
    precision = precision_score(y_val, probs_class) 
    recall    = recall_score(y_val, probs_class)
    fscore    = f1_score(y_val, probs_class)

    fsc.append(fscore)
    print(f' fscore {fscore}')
    
    del model
    K.clear_session()
    gc.collect()

 fold 0


INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 400)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 400, 768),)       66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 66,363,649
Trainable params: 66,363,649
Non-trainable params: 0
_________________________________________________________________
Fitting
496/496 [==============================] - 23s 47ms/step
 average precision 0.8105385856733817
 roc auc 0.96915976635651
 fscore 0.24244090159428253


### CV Results

In [16]:
np.array(auc).mean()

0.8856419362673953

In [17]:
np.array(auc).std()

0.035024733300714865

In [18]:
np.array(roc).mean()

0.9778657783483876

In [19]:
np.array(roc).std()

0.004460430339877984

In [20]:
np.array(fsc).mean()

0.7375734118482549

In [21]:
np.array(fsc).std()

0.18954725075888815

In [22]:
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.222.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [23]:
x_train = fast_encode(X, fast_tokenizer, maxlen=400)
x_test = fast_encode(X_test, fast_tokenizer, maxlen=400)

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y))
    .shuffle(42)
    .batch(64)
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test, y_test))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

In [24]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    model = build_model(transformer_layer, max_len=400)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 400)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 400, 768),)       66362880  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 66,363,649
Trainable params: 66,363,649
Non-trainable params: 0
_________________________________________________________________
CPU times: user 5.73 s, sys: 2.55 s, total: 8.27 s
Wall time: 11.6 s


In [0]:
#model.layers[1].trainable = False

In [26]:
if TRAIN:
  train_history = model.fit(
      train_dataset,
      #validation_data=test_dataset,
      epochs=1
  )
  model.save_weights(f'{models_path}/DistilBERT_full.h5')
else:
  model.load_weights(f'{models_path}/DistilBERT_full.h5')

2480/2480 [==============================] - 326s 131ms/step - loss: 0.1180 - auc: 0.9685


In [27]:
probs = model.predict(test_dataset, batch_size=64, verbose=1)



1013/1013 [==============================] - 48s 47ms/step


In [0]:
threshold = 0.1
probs_class = probs.copy()
probs_class[probs_class >= threshold] = 1 
probs_class[probs_class < threshold] = 0

In [0]:
auc_f   = average_precision_score(y_test, probs)
roc_f   = roc_auc_score(y_test, probs)
f_score = f1_score(y_test, probs_class)

In [30]:
auc_f

0.8666090215834295

In [31]:
roc_f

0.9754533141454953

In [32]:
f_score

0.7772772390623808